In [2]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import Fuzzy.mm_utils
from Fuzzy.function_util import *
from Fuzzy.IMP import IMP
from Fuzzy.SMP1 import SMP1
from Fuzzy.SMP2 import SMP2
from Fuzzy.FIS1 import FIS1
from Fuzzy.FIS2 import FIS2
from Fuzzy.FIS3 import FIS3

ModuleNotFoundError: No module named 'mm_utils'

In [8]:
# load all the data 
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')

# select error bound 
err_size = 38
# select cut off point for smp1, for use 60 following Quddus paper
cutoff = 60 
# select number of no match needed to break iteration 
no_match_max = 3

# intialize no match counter 
no_match = 0 
# initialize matched trajectory list
matched_point_list = []
# initialize matched edge list 
matched_edge_list = []
# initialize SMP1 result list 
res_list =[]
# initialize counter for matched_list 
count = 0

# Run IMP Algorithm to find first point 
point_index, curr_edge = IMP(gdf_utm, edges_utm, 2)

#store current edge 
matched_edge_list.append(curr_edge)

# matched current location with current edge 
curr_loc = gdf_utm.iloc[[point_index]]
matched_point = point_matching(curr_loc, curr_edge.iloc[0])
# find longitude and latitude for last matched data
matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})

# update previous matched position
last_matched = matched_point

# store matched point 
matched_point_list.append(matched_point)

# move to the next iteration with for loop
start_iter = point_index + 1


for point_index in range(start_iter, len(gdf_utm)):    
    print(point_index)
    # update last location 
    prev_loc = gdf_utm.iloc[[point_index - 1]]

    # update current location 
    curr_loc = gdf_utm.iloc[[point_index]]
    
    # perform SMP1 
    res = SMP1(curr_loc, curr_edge, prev_loc, last_matched, nodes_utm, edges_utm, gdf_utm)
    # store result list
    res_list.append(res)

    # decide whether vehicle is entering junction or not cut off 
    if res >= cutoff:
        # vehicle is not entering a new edge, matched edge doesnt change
        # update matched edge list 
        matched_edge_list.append(curr_edge)
    elif res < cutoff:
        prev_edge = curr_edge
        # SMP1 detect vehicle is entering new edge, perform SMP 2 to update current edge
        edge_candidate = SMP2(curr_loc, curr_edge, prev_loc, last_matched, err_size, nodes_utm, edges_utm, gdf_utm)
        
        if edge_candidate is None:
            print(['No edge can be matched at trajectory number', point_index])
            no_match += 1
        else:
            curr_edge = edge_candidate
            # curr_edge['prev_end_node'] = prev_edge.index[0][1]
            # there is no more matched edge, break the loop 
            matched_edge_list.append(curr_edge)
            # update matched point
            matched_point = point_matching(curr_loc, curr_edge.iloc[0])
            # find longitude and latitude for last matched data
            matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
            # store matched point 
            matched_point_list.append(matched_point)
            # update previous matched position
            last_matched = matched_point
    
    if no_match == no_match_max:
        # stop iteration when there number of no match is exceeded
        break

    # update count 
    count +=  1

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 1]
['no edeges intersects with error bound at iteration number', 2]
['no edeges intersects with error bound at iteration number', 3]
['no edeges intersects with error bound at iteration number', 4]
['no edeges intersects with error bound at iteration number', 5]
['no edeges intersects with error bound at iteration number', 6]
['no edeges intersects with error bound at iteration number', 7]
['no edeges intersects with error bound at iteration number', 8]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 9]
['no edeges intersects with error bound at iteration number', 10]
['no edeges intersects with error bound at iteration number', 11]
['edges found at iteration number', 12]
['edges found at iteration number', 13]
['stopping at position', 12]
13


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\U

...

50
51
52
53
54
55
56


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPytho

57
58
['No edge can be matched at trajectory number', 58]
59
['No edge can be matched at trajectory number', 59]
60


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\U

['No edge can be matched at trajectory number', 60]


In [17]:
# create dataframe for the results
matched_edge_df = pd.concat(matched_edge_list)
matched_point_df = pd.concat(matched_point_list)

In [16]:
# measure connectivity of the link 
matched_edge_df

osmid  \
u          v          key                                                   
135535395  53200191   0                            [673214248, 234046511]   
                      0                            [673214248, 234046511]   
53200191   6152925224 0                                         470244584   
6152925224 53200192   0                            [673214250, 467653690]   
53200192   256534920  0                                         148065615   
256534920  53200193   0                                         148065615   
                      0                                         148065615   
53200193   6152925223 0                                         148065615   
6152925223 43599624   0                            [234046510, 148065615]   
                      0                            [234046510, 148065615]   
                      0                            [234046510, 148065615]   
43599624   6152925223 0                            [234046510, 148065615]   
                      0                            [234046510, 148065615]   
           110368894  0                            [929169177, 665654366]   
                      0                            [929169177, 665654366]   
110368894  52139627   0                            [929169177, 160290311]   
52139627   83679375   0                                         160290311   
                      0                                         160290311   
83679375   43600642   0                                         160290311   
43600642   91718741   0                                         160290311   
91718741   81331484   0                                         160290311   
81331484   51516875   0                                         320646340   
                      0                                         320646340   
                      0                                         320646340   
                      0                                         320646340   
                      0                                         320646340   
51516875   296965817  0                            [727368840, 147518679]   
496537326  1631144431 0    [570314775, 147392396, 1024533276, 1024533279]   
                      0    [570314775, 147392396, 1024533276, 1024533279]   
1631144431 314436395  0                                         160290370   
314436395  42907691   0                                         207295803   
42907691   1504411732 0                                          31773344   
37513773   42908014   0                                           5707707   
                      0                                           5707707   
                      0                                           5707707   
42908014   2505759277 0                             [569973236, 31770303]   
2505759277 2505759276 0                                         243147188   
                      0                                         243147188   
                      0                                         243147188   
                      0                                         243147188   
42907801   37514058   0        [243147186, 5608044, 293180274, 569951386]   
                      0        [243147186, 5608044, 293180274, 569951386]   
                      0        [243147186, 5608044, 293180274, 569951386]   
                      0        [243147186, 5608044, 293180274, 569951386]   
                      0        [243147186, 5608044, 293180274, 569951386]   
37514058   41665100   0                                           5707696   

                            lanes    ref               name  \
u          v          key                                     
135535395  53200191   0         3  L 793   Wolbecker Straße   
                      0         3  L 793   Wolbecker Straße   
53200191   6152925224 0         3  L 793   Wolbecker Straße   
6152925224 53200192   0         2  L 793   Wolbecker Str

# Visualize Result 

In [18]:
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = gdf_utm['geometry'].to_frame()
temp = point_locs.plot(ax = ax)

# this code plot all the road system 
edges_utm.plot(ax=ax)

# matched point plot
matched_point_df.plot(ax = ax, color = "Green")

# matched_edge 
matched_edge_df.plot(ax = ax, color = "Black")


<Axes: >

In [44]:
gdf_utm['phenomenons'].dict

AttributeError: 'Series' object has no attribute 'dict'